In [2]:
from datasets import load_dataset
ds = load_dataset("brimmann2/squad-v2-sampled")

In [3]:
import torch
model_name = "Hannibal046/xrag-7b"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [6]:
from transformers import AutoConfig, AutoTokenizer
from models.modeling_xmistral import XMistralForCausalLM

config = AutoConfig.from_pretrained(model_name)
MODEL_CLASS = eval(config.architectures[0])
xmistral = MODEL_CLASS.from_pretrained(
    model_name,
    torch_dtype = torch.bfloat16,
    low_cpu_mem_usage = True,
).to(device)

xmistral_tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
XRAG_TOKEN = "<xRAG>" 
xmistral.set_xrag_token_id(xmistral_tokenizer.encode(XRAG_TOKEN))

In [14]:
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig
from configs.fsdp import fsdp_config as FsdpConfig

In [15]:
dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
fsdp_config = DistillationConfig()
train_config.model_name = "google/gemma-3-1b-it"
distill_config.model_name = "Hannibal046/xrag-7b"
distill_config.xrag = True
train_config.batch_size_training = 1
train_config.num_workers_dataloader = 1
dataset_config.file = "datasets_m/loader/squad.py"
dataset_config.xrag = True

In [17]:
from data.data_utils import get_dataloader
student_train_dataloader, student_eval_dataloader = get_dataloader(dataset_config, train_config, xmistral_tokenizer, 0, distill_config)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

--> Training Set Length = 1000


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

--> Validation Set Length = 250


In [23]:
iter1 = iter(student_train_dataloader)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
a = next(iter1)

In [28]:
input_ids, attention_mask, retrieval_embeds, labels = a

In [ ]:
with torch.no_grad():
    output = xmistral(input_ids=input_ids, attention_mask=attention_mask, retrieval_embeds=retrieval_embeds, labels=labels)

TypeError: forward() takes from 1 to 4 positional arguments but 5 were given